# Segmenting and Clustering
## Neighborhoods in Toronto City
### Parsing data from Wikipedia page, and creating a dataframe.

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [3]:
url_w = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
shtml = requests.get(url_w).text
soup = BeautifulSoup(shtml, 'html.parser')
table = soup.table
soup.find('table')
print("... table data parsed ...")

... table data parsed ...


In [4]:
table_rows = table.find_all('tr')
rows_list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    rows_list.append(row)
    print(row)

[]
['M1A', 'Not assigned', 'Not assigned\n']
['M2A', 'Not assigned', 'Not assigned\n']
['M3A', 'North York', 'Parkwoods\n']
['M4A', 'North York', 'Victoria Village\n']
['M5A', 'Downtown Toronto', 'Harbourfront\n']
['M5A', 'Downtown Toronto', 'Regent Park\n']
['M6A', 'North York', 'Lawrence Heights\n']
['M6A', 'North York', 'Lawrence Manor\n']
['M7A', "Queen's Park", 'Not assigned\n']
['M8A', 'Not assigned', 'Not assigned\n']
['M9A', 'Etobicoke', 'Islington Avenue\n']
['M1B', 'Scarborough', 'Rouge\n']
['M1B', 'Scarborough', 'Malvern\n']
['M2B', 'Not assigned', 'Not assigned\n']
['M3B', 'North York', 'Don Mills North\n']
['M4B', 'East York', 'Woodbine Gardens\n']
['M4B', 'East York', 'Parkview Hill\n']
['M5B', 'Downtown Toronto', 'Ryerson\n']
['M5B', 'Downtown Toronto', 'Garden District\n']
['M6B', 'North York', 'Glencairn\n']
['M7B', 'Not assigned', 'Not assigned\n']
['M8B', 'Not assigned', 'Not assigned\n']
['M9B', 'Etobicoke', 'Cloverdale\n']
['M9B', 'Etobicoke', 'Islington\n']
['M9B'

In [5]:
rows_list[0:5]
df_neigh = pd.DataFrame(rows_list)
df_neigh.head(5)

,0,1,2
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [6]:
df_neigh.columns = ['PostalCode', 'Borough','Neighborhood']
df_neigh.drop(0, inplace = True)
df_neigh.drop(df_neigh.loc[df_neigh['Borough']=='Not assigned'].index, inplace=True)
df_neigh.head(5)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n
6,M5A,Downtown Toronto,Regent Park\n
7,M6A,North York,Lawrence Heights\n


In [7]:
df_neigh['Neighborhood'] = df_neigh['Neighborhood'].map(lambda x: x.rstrip('\n'))
df_neigh.head(5)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [8]:
df_tor = df_neigh.astype(str).groupby('PostalCode').agg(lambda x: ','.join(x.unique()))
df_tor.reset_index(inplace = True) 
df_tor.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df_tor['Neighborhood'].replace("Not assigned", np.nan, inplace = True)
df_tor.Neighborhood.fillna(df_tor.Borough, inplace=True)
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [11]:
df_tor.to_csv('Toronto_PostalCodes.csv')
toronto_set = df_tor.sort_values(by=['Neighborhood'])
toronto_set.head(10)

,PostalCode,Borough,Neighborhood
58,M5H,Downtown Toronto,"Adelaide,King,Richmond"
12,M1S,Scarborough,Agincourt
14,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
89,M8W,Etobicoke,"Alderwood,Long Branch"
28,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights"
19,M2K,North York,Bayview Village
62,M5M,North York,"Bedford Park,Lawrence Manor East"
56,M5E,Downtown Toronto,Berczy Park
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [12]:
toronto_set.shape

(103, 3)

## Importing Geospatial data for Neighborhoods:

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('tbody')
rows = table.select('tr')
data = [r.get_text() for r in rows]
df = pd.DataFrame(data)

df = df[0].str.split('\n', expand=True)
df = df.rename(columns=df.iloc[0])
df = df.drop(df.index[0])

df = df[df.Borough != 'Not assigned']

df = df.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df.reset_index(inplace = True)

df['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)


In [23]:
df_cord = pd.read_csv("https://cocl.us/Geospatial_data")
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df_tor.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [26]:
df_cord.shape

(103, 3)

In [27]:
df_tor.shape

(103, 3)

In [28]:
df_toronto = pd.concat([df_tor, df_cord], axis = 1)
df_toronto.head(10)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",M1N,43.692657,-79.264848


In [29]:
df_toronto.tail(10)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
93,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",M9B,43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",M9C,43.643515,-79.577201
96,M9L,North York,Humber Summit,M9L,43.756303,-79.565963
97,M9M,North York,"Emery,Humberlea",M9M,43.724766,-79.532242
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",M9R,43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V,43.739416,-79.588437
102,M9W,Etobicoke,Northwest,M9W,43.706748,-79.594054


In [30]:
df_toronto.drop('Postal Code', axis=1, inplace = True)
df_toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [31]:
df_toronto.to_csv('Toronto_FullSet.csv')

## Segmenting and Clustering Neighborhoods in Toronto

In [33]:

import numpy as np
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.6.16          |           py36_0         148 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [34]:
df_toronto.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [35]:

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [36]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [37]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [38]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Scarborough, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Scarborough, Toronto are 43.773077, -79.257774.


In [39]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [45]:
CLIENT_ID = 'YXUBUWGZ5JHXV2UCFP53OCMFP5JUCV5RQTT4P5XIUGZDTWNX' # your Foursquare ID
CLIENT_SECRET = 'ZCPH1BZUNIWU0BEFXKA25JUMDHKNOJUNER4OOWAF51POLGGX' # your Foursquare Secret
VERSION = '20190623' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YXUBUWGZ5JHXV2UCFP53OCMFP5JUCV5RQTT4P5XIUGZDTWNX
CLIENT_SECRET:ZCPH1BZUNIWU0BEFXKA25JUMDHKNOJUNER4OOWAF51POLGGX


In [41]:

scarborough_data.loc[4, 'Neighborhood']

'Cedarbrae'

In [42]:
neighborhood_latitude = scarborough_data.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cedarbrae are 43.773136, -79.23947609999999.


In [46]:
radius = 600 # define radius
LIMIT = 300
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YXUBUWGZ5JHXV2UCFP53OCMFP5JUCV5RQTT4P5XIUGZDTWNX&client_secret=ZCPH1BZUNIWU0BEFXKA25JUMDHKNOJUNER4OOWAF51POLGGX&v=20190623&ll=43.773136,-79.23947609999999&radius=600&limit=300'

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2b0f54531593002c6277ba'},
 'response': {'headerLocation': 'Woburn',
  'headerFullLocation': 'Woburn, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 43.77853600540001,
    'lng': -79.23201170809249},
   'sw': {'lat': 43.767735994599995, 'lng': -79.2469404919075}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b1711a6f964a520cbc123e3',
       'name': 'Federick Restaurant',
       'location': {'address': '1920 Ellesmere Rd',
        'crossStreet': 'at Bellamy Rd. N',
        'lat': 43.77469659057996,
        'lng': -79.24114242818267,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.77469659057996,
          'lng': -79.24114242818267}],
        'd

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
1,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
2,CANBE Foods Inc,Indian Restaurant,43.773546,-79.246082
3,Thai One On,Thai Restaurant,43.774468,-79.241268
4,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500
5,Hakka Legend,Chinese Restaurant,43.776309,-79.234939
6,La Sani Grill,Indian Restaurant,43.776214,-79.234848
7,TD Canada Trust,Bank,43.774952,-79.241343
8,B&A Bakery,Bakery,43.774391,-79.243877
9,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.775930,-79.235328


In [50]:
print('{} venues were returned by Foursquare. Due to given Limit'.format(nearby_venues.shape[0]))

14 venues were returned by Foursquare. Due to given Limit


## Exploring Neighborhoods in Scarborough, Toronto:

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


In [53]:
print(scarborough_venues.shape)
scarborough_venues.head()

(144, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,"Rouge,Malvern",43.806686,-79.194353,Lee Valley,43.803161,-79.199681,Hobby Shop
4,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course


In [54]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,6,6,6,6,6,6
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",7,7,7,7,7,7
"Birch Cliff,Cliffside West",5,5,5,5,5,5
Cedarbrae,14,14,14,14,14,14
"Clairlea,Golden Mile,Oakridge",16,16,16,16,16,16
"Clarks Corners,Sullivan,Tam O'Shanter",14,14,14,14,14,14
"Cliffcrest,Cliffside,Scarborough Village West",6,6,6,6,6,6
"Dorset Park,Scarborough Town Centre,Wexford Heights",11,11,11,11,11,11
"East Birchmount Park,Ionview,Kennedy Park",10,10,10,10,10,10


## Neighborhood Analysis

In [55]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,ATM,American Restaurant,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Business Service,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Electronics Store,Fast Food Restaurant,Fish Market,Flower Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Rental Car Location,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Wings Joint,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"Rouge,Malvern"
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge,Malvern"
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge,Malvern"
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge,Malvern"
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Highland Creek,Rouge Hill,Port Union"


In [56]:
scarborough_onehot.shape

(144, 71)

In [57]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,ATM,American Restaurant,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Business Service,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Electronics Store,Fast Food Restaurant,Fish Market,Flower Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Rental Car Location,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Wings Joint
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.166667,0.166667,0.0,0.0000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.000000,0.000000,0.000000,0.285714,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.0000,0.0,0.000,0.0,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0000,0.0,0.200,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000
3,Cedarbrae,0.000000,0.000000,0.071429,0.071429,0.000000,0.142857,0.071429,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.071429,0.071429,0.000000,0.071429,0.0,0.0000,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.071429,0.071429,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.071429,0.00,0.142857,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.071429,0.000000,0.000000,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.0,0.000000,0.000000,0.125000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.125000,0.0,0.0625,0.0,0.125,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.187500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.00,0.000000,0.000000,0.000000,0.000000
5,"Clarks Corners,Sull

In [58]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0     Coffee Shop  0.17
1  Breakfast Spot  0.17
2  Clothing Store  0.17
3  Sandwich Place  0.17
4    Skating Rink  0.17


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                  venue  freq
0             BBQ Joint  0.29
1  Fast Food Restaurant  0.29
2                  Park  0.14
3                Bakery  0.14
4    Chinese Restaurant  0.14


----Birch Cliff,Cliffside West----
                   venue  freq
0        College Stadium   0.2
1                  Diner   0.2
2  General Entertainment   0.2
3           Skating Rink   0.2
4                   Café   0.2


----Cedarbrae----
                  venue  freq
0                Bakery  0.14
1     Indian Restaurant  0.14
2      Hakka Restaurant  0.07
3  Caribbean Restaurant  0.07
4      Asian Restaurant  0.07


----Clairlea,Golden Mile,Oakridge----
          venue  freq
0  Intersection  0.19
1      Bus Line  0.12
2         Diner  0.12
3        Bakery  0.12
4   Coffee Shop  0.12

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Coffee Shop,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Sandwich Place,College Stadium,Convenience Store,Department Store,Diner
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",BBQ Joint,Fast Food Restaurant,Chinese Restaurant,Park,Bakery,Electronics Store,College Stadium,Convenience Store,Department Store,Diner
2,"Birch Cliff,Cliffside West",General Entertainment,College Stadium,Café,Skating Rink,Diner,Wings Joint,Electronics Store,Convenience Store,Department Store,Discount Store
3,Cedarbrae,Bakery,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant,Flower Shop,Fried Chicken Joint,Bank,Coffee Shop,Athletics & Sports,Asian Restaurant
4,"Clairlea,Golden Mile,Oakridge",Intersection,Diner,Bus Line,Coffee Shop,Bakery,Fast Food Restaurant,Convenience Store,Park,Metro Station,Soccer Field


## Cluster Neighborhoods

In [60]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [61]:

neighborhoods_venues_sorted.insert(0, 'Clusters', kmeans.labels_)

scarborough_merged = scarborough_data

scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


scarborough_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Hobby Shop,Spa,Wings Joint,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Electronics Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4.0,Golf Course,Bar,Wings Joint,Fast Food Restaurant,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Electronics Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2.0,Electronics Store,Breakfast Spot,Park,Bus Line,Intersection,Pizza Place,Medical Center,Rental Car Location,Fast Food Restaurant,Mexican Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Insurance Office,Business Service,Electronics Store,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Wings Joint
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Bakery,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant,Flower Shop,Fried Chicken Joint,Bank,Coffee Shop,Athletics & Sports,Asian Restaurant


In [62]:
scarborough_merged.dropna(axis=0,inplace = True)
scarborough_merged['Clusters'] = scarborough_merged['Clusters'].astype('int')
scarborough_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Hobby Shop,Spa,Wings Joint,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Electronics Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4,Golf Course,Bar,Wings Joint,Fast Food Restaurant,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Electronics Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2,Electronics Store,Breakfast Spot,Park,Bus Line,Intersection,Pizza Place,Medical Center,Rental Car Location,Fast Food Restaurant,Mexican Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Insurance Office,Business Service,Electronics Store,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Wings Joint
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Bakery,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant,Flower Shop,Fried Chicken Joint,Bank,Coffee Shop,Athletics & Sports,Asian Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3,Pizza Place,Middle Eastern Restaurant,Convenience Store,Clothing Store,Coffee Shop,College Stadium,Department Store,Diner,Discount Store,Wings Joint
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,2,Discount Store,Bus Line,Convenience Store,Hobby Shop,Metro Station,Department Store,Light Rail Station,Bus Station,Coffee Shop,Wings Joint
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,2,Intersection,Diner,Bus Line,Coffee Shop,Bakery,Fast Food Restaurant,Convenience Store,Park,Metro Station,Soccer Field
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,2,Wings Joint,Chinese Restaurant,Furniture / Home Store,Motel,Movie Theater,American Restaurant,BBQ Joint,Coffee Shop,General Entertainment,Gas Station
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,2,General Entertainment,College Stadium,Café,Skating Rink,Diner,Wings Joint,Electronics Store,Convenience Store,Department Store,Discount Store


In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'],scarborough_merged['Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [64]:
scarborough_merged.loc[scarborough_merged['Clusters'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Hobby Shop,Spa,Wings Joint,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Electronics Store
14,Scarborough,0,BBQ Joint,Fast Food Restaurant,Chinese Restaurant,Park,Bakery,Electronics Store,College Stadium,Convenience Store,Department Store,Diner


In [65]:
scarborough_merged.loc[scarborough_merged['Clusters'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,1,Coffee Shop,Insurance Office,Business Service,Electronics Store,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Wings Joint


In [66]:
scarborough_merged.loc[scarborough_merged['Clusters'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,2,Electronics Store,Breakfast Spot,Park,Bus Line,Intersection,Pizza Place,Medical Center,Rental Car Location,Fast Food Restaurant,Mexican Restaurant
4,Scarborough,2,Bakery,Indian Restaurant,Chinese Restaurant,Caribbean Restaurant,Flower Shop,Fried Chicken Joint,Bank,Coffee Shop,Athletics & Sports,Asian Restaurant
6,Scarborough,2,Discount Store,Bus Line,Convenience Store,Hobby Shop,Metro Station,Department Store,Light Rail Station,Bus Station,Coffee Shop,Wings Joint
7,Scarborough,2,Intersection,Diner,Bus Line,Coffee Shop,Bakery,Fast Food Restaurant,Convenience Store,Park,Metro Station,Soccer Field
8,Scarborough,2,Wings Joint,Chinese Restaurant,Furniture / Home Store,Motel,Movie Theater,American Restaurant,BBQ Joint,Coffee Shop,General Entertainment,Gas Station
9,Scarborough,2,General Entertainment,College Stadium,Café,Skating Rink,Diner,Wings Joint,Electronics Store,Convenience Store,Department Store,Discount Store
10,Scarborough,2,Indian Restaurant,Electronics Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Bakery,Pet Store,Coffee Shop,Light Rail Station,Convenience Store
11,Scarborough,2,Grocery Store,Seafood Restaurant,Vietnamese Restaurant,Fish Market,Intersection,Korean Restaurant,Coffee Shop,Pizza Place,Breakfast Spot,Gas Station
12,Scarborough,2,Coffee Shop,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Sandwich Place,College Stadium,Convenience Store,Department Store,Diner
13,Scarborough,2,Pizza Place,Pharmacy,ATM,Noodle House,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Shopping Mall,Bank,Sandwich Place


In [67]:
scarborough_merged.loc[scarborough_merged['Clusters'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3,Pizza Place,Middle Eastern Restaurant,Convenience Store,Clothing Store,Coffee Shop,College Stadium,Department Store,Diner,Discount Store,Wings Joint


In [68]:
scarborough_merged.loc[scarborough_merged['Clusters'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4,Golf Course,Bar,Wings Joint,Fast Food Restaurant,College Stadium,Convenience Store,Department Store,Diner,Discount Store,Electronics Store
